<h1>Extract participant video from zoom recording</h1>

In [15]:
import subprocess

# Cut the video
input_video = '001.mp4'
start_time = '00:52:38'  
end_time = '00:58:29'    
output_cut = '001_shorter.mp4'

cut_command = f"ffmpeg -i {input_video} -ss {start_time} -to {end_time} -c copy {output_cut}"
subprocess.run(cut_command, shell=True)

# Extract Face Video
output_cropped = 'output_cropped.mp4'

crop_command = f"ffmpeg -i {output_cut} -analyzeduration 10M -probesize 10M -c:v libx264 -vf 'crop=640:480:10:10' {output_cropped}"

# Run the command and capture output
result = subprocess.run(crop_command, shell=True, capture_output=True, text=True)

# Print the output
print("FFmpeg Output:", result.stdout)

# Check for errors
if result.returncode != 0:
    print("Error:", result.stderr)


FFmpeg Output: 
Error: ffmpeg version N-113801-g6c83cd5cb3-20240224 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (crosstool-NG 1.25.0.232_c175b21)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libaribcaption --enable-libass --enable-libbluray --enable-libjxl --enable-li

In [22]:
import cv2

# Input and output video filenames
input_video_filename = '001_shorter.mp4'
output_cropped_filename = '001_cropped.mp4'

# Open the video file
cap = cv2.VideoCapture(input_video_filename)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set the desired crop region (adjust these values as needed)
x, y, w, h = 1650, 200, 200, 180

# Create a VideoWriter object to write the cropped video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec if needed
out = cv2.VideoWriter(output_cropped_filename, fourcc, 30, (w, h))

# Read and write frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Crop the frame
    cropped_frame = frame[y:y+h, x:x+w]

    # Write the cropped frame to the output video
    out.write(cropped_frame)

# Release the video capture and writer objects
cap.release()
out.release()

# Destroy any OpenCV windows
cv2.destroyAllWindows()
